In [1]:
!pip install git+https://github.com/afnan47/cuda.git

  Cloning https://github.com/afnan47/cuda.git to /tmp/pip-req-build-x8ttj36s
  Running command git clone --filter=blob:none --quiet https://github.com/afnan47/cuda.git /tmp/pip-req-build-x8ttj36s
  Resolved https://github.com/afnan47/cuda.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4289 sha256=2e335fe535b85f8470205fbc8db6dcd11a8c5f06dab758fe3786ee90d0177865
  Stored in directory: /tmp/pip-ephem-wheel-cache-h1gsyrxm/wheels/aa/f3/44/e10c1d226ec561d971fcd4b0463f6bff08602afa928a3e7bc7
Successfully built NVCCPlugin


In [2]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [3]:
%%writefile ass3.cu

#include <iostream>

__global__ void minKernel(int *arr, int *minval, int n) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid < n) atomicMin(minval, arr[tid]);
}

__global__ void maxKernel(int *arr, int *maxval, int n) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid < n) atomicMax(maxval, arr[tid]);
}

__global__ void sumKernel(int *arr, int *sum, int n) {
    int tid = threadIdx.x + blockIdx.x * blockDim.x;
    if (tid < n) atomicAdd(sum, arr[tid]);
}

int minval(int arr[], int n) {
    int *d_arr, *d_minval;
    cudaMalloc(&d_arr, n * sizeof(int));
    cudaMalloc(&d_minval, sizeof(int));
    cudaMemcpy(d_arr, arr, n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_minval, &arr[0], sizeof(int), cudaMemcpyHostToDevice);

    minKernel<<<(n + 255) / 256, 256>>>(d_arr, d_minval, n);

    int minval;
    cudaMemcpy(&minval, d_minval, sizeof(int), cudaMemcpyDeviceToHost);

    cudaFree(d_arr);
    cudaFree(d_minval);

    return minval;
}

int maxval(int arr[], int n) {
    int *d_arr, *d_maxval;
    cudaMalloc(&d_arr, n * sizeof(int));
    cudaMalloc(&d_maxval, sizeof(int));
    cudaMemcpy(d_arr, arr, n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_maxval, &arr[0], sizeof(int), cudaMemcpyHostToDevice);

    maxKernel<<<(n + 255) / 256, 256>>>(d_arr, d_maxval, n);

    int maxval;
    cudaMemcpy(&maxval, d_maxval, sizeof(int), cudaMemcpyDeviceToHost);

    cudaFree(d_arr);
    cudaFree(d_maxval);

    return maxval;
}

int sum(int arr[], int n) {
    int *d_arr, *d_sum;
    cudaMalloc(&d_arr, n * sizeof(int));
    cudaMalloc(&d_sum, sizeof(int));
    cudaMemcpy(d_arr, arr, n * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_sum, &arr[0], sizeof(int), cudaMemcpyHostToDevice);

    sumKernel<<<(n + 255) / 256, 256>>>(d_arr, d_sum, n);

    int sum;
    cudaMemcpy(&sum, d_sum, sizeof(int), cudaMemcpyDeviceToHost);

    cudaFree(d_arr);
    cudaFree(d_sum);

    return sum;
}

int average(int arr[], int n) {
    return (double)sum(arr, n) / n;
}

int main() {
    int n = 5;
    int arr[] = {1, 2, 3, 4, 5};

    std::cout << "The minimum value is: " << minval(arr, n) << '\n';
    std::cout << "The maximum value is: " << maxval(arr, n) << '\n';
    std::cout << "The summation is: " << sum(arr, n) << '\n';
    std::cout << "The average is: " << average(arr, n) << '\n';

    return 0;
}

Writing ass3.cu


In [4]:
!nvcc -o ass3 ass3.cu -Xcompiler -fopenmp
!./ass3

The minimum value is: 1
The maximum value is: 5
The summation is: 16
The average is: 3
